In [40]:
import pandas as pd
import voila


# Read data from JSON file
df = pd.read_json('project_3.fridge_fills.json')

# Drop the '_id' column if it exists
df.drop('_id', axis=1, inplace=True, errors='ignore')

df.head()


,Date,Day of Week,Time of Day,Name,Pickup Location,Dropoff Location,Fill Type
0,{'$date': '2023-01-01T05:00:00.000Z'},Sunday,Morning (7AM-12PM),Krista Kircanski,GO - Grocery Outlet,BOK,Rescued Groceries
1,{'$date': '2023-01-02T05:00:00.000Z'},Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,BOK,Purchased Groceries
2,{'$date': '2023-01-02T05:00:00.000Z'},Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,DS - DiSilvestro,Purchased Groceries
3,{'$date': '2023-01-02T05:00:00.000Z'},Monday,Evening (5PM-10PM),Kathryn Nolan,Grocery FILL,BOK,Purchased Groceries
4,{'$date': '2023-01-02T05:00:00.000Z'},Monday,Morning (7AM-12PM),Leah Santos,Grocery FILL,Mifflin,Purchased Groceries


In [41]:
import pandas as pd

# Read data from JSON file
df = pd.read_json('project_3.fridge_fills.json')

# Extract the date values from the '$date' key
df['Date'] = df['Date'].apply(lambda x: x['$date'])

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Drop the '_id' column if it exists
df.drop('_id', axis=1, inplace=True, errors='ignore')

df.head()


,Date,Day of Week,Time of Day,Name,Pickup Location,Dropoff Location,Fill Type
0,2023-01-01 05:00:00+00:00,Sunday,Morning (7AM-12PM),Krista Kircanski,GO - Grocery Outlet,BOK,Rescued Groceries
1,2023-01-02 05:00:00+00:00,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,BOK,Purchased Groceries
2,2023-01-02 05:00:00+00:00,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,DS - DiSilvestro,Purchased Groceries
3,2023-01-02 05:00:00+00:00,Monday,Evening (5PM-10PM),Kathryn Nolan,Grocery FILL,BOK,Purchased Groceries
4,2023-01-02 05:00:00+00:00,Monday,Morning (7AM-12PM),Leah Santos,Grocery FILL,Mifflin,Purchased Groceries


In [42]:
import pandas as pd

# Read data from JSON file
df = pd.read_json('project_3.fridge_fills.json')

# Extract the date values from the '$date' key
df['Date'] = df['Date'].apply(lambda x: x['$date'])

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Format the dates to display in the desired format
df['Date'] = df['Date'].dt.strftime('%m-%d-%Y')

# Drop the '_id' column if it exists
df.drop('_id', axis=1, inplace=True, errors='ignore')

df.head()


,Date,Day of Week,Time of Day,Name,Pickup Location,Dropoff Location,Fill Type
0,01-01-2023,Sunday,Morning (7AM-12PM),Krista Kircanski,GO - Grocery Outlet,BOK,Rescued Groceries
1,01-02-2023,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,BOK,Purchased Groceries
2,01-02-2023,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,DS - DiSilvestro,Purchased Groceries
3,01-02-2023,Monday,Evening (5PM-10PM),Kathryn Nolan,Grocery FILL,BOK,Purchased Groceries
4,01-02-2023,Monday,Morning (7AM-12PM),Leah Santos,Grocery FILL,Mifflin,Purchased Groceries


In [43]:
# import pandas as pd
# from pymongo import MongoClient
# import matplotlib.pyplot as plt

# client = MongoClient('localhost', 27017)  
# db = client.project_3  
# collection = db.fridge_fills 

# cursor = collection.find({})  

# data = list(cursor)

# df = pd.DataFrame(data)

# df.drop('_id', axis=1, inplace=True)

# df.head()


In [44]:
# df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x).date())
# df_dates = df

# # Display the DataFrame
# df_dates.head()

In [39]:
import pandas as pd
from ipywidgets import interact, widgets
import plotly.express as px

# Convert 'Date' column to datetime
df_dates['Date'] = pd.to_datetime(df_dates['Date'])

# Extract month and year from 'Date' column
df_dates['YearMonth'] = df_dates['Date'].dt.to_period('M').astype(str)  

def plot_entries_per_month(fill_type):
    if fill_type == 'All Fill Types':
        filtered_df = df_dates.copy()  
    else:
        filtered_df = df_dates[df_dates['Fill Type'] == fill_type].copy()  

    filtered_df.loc[:, 'YearMonth'] = filtered_df['Date'].dt.to_period('M').astype(str)  

    monthly_counts = filtered_df.groupby('YearMonth').size().reset_index(name='Total Entries')

    # Set figure size
    fig = px.bar(monthly_counts, x='YearMonth', y='Total Entries', 
                 labels={'YearMonth': 'Month', 'Total Entries': 'Total Entries'},
                 title=f'Total Fills per Month for {fill_type}')
    fig.update_layout(xaxis_tickangle=-45, yaxis_gridcolor='lightgrey', 
                      width=800, height=500) 
    fig.show()

# Get unique values of 'Fill Type' for dropdown menu excluding 'nan'
fill_types = df_dates['Fill Type'].dropna().unique()
fill_types = list(fill_types)
fill_types.append('All Fill Types')  

# Create interactive dropdown menu
interact(plot_entries_per_month, fill_type=widgets.Dropdown(options=fill_types, description='Fill Type:'))


interactive(children=(Dropdown(description='Fill Type:', options=('Rescued Groceries', 'Purchased Groceries', …

<function __main__.plot_entries_per_month(fill_type)>

In [30]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Load your data
# Assuming df_dates is your DataFrame

# Convert 'Date' column to datetime
df_dates['Date'] = pd.to_datetime(df_dates['Date'])

# Extract month and year from 'Date' column
df_dates['YearMonth'] = df_dates['Date'].dt.to_period('M').astype(str)  

# Get unique values of 'Fill Type' for dropdown menu excluding 'nan'
fill_types = df_dates['Fill Type'].dropna().unique()
fill_types = list(fill_types)
fill_types.append('All Fill Types')  

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Total Fills per Month"),
    dcc.Dropdown(
        id='fill-type-dropdown',
        options=[{'label': ft, 'value': ft} for ft in fill_types],
        value='All Fill Types'
    ),
    dcc.Graph(id='fill-type-bar-chart')
])

@app.callback(
    Output('fill-type-bar-chart', 'figure'),
    [Input('fill-type-dropdown', 'value')]
)
def update_bar_chart(fill_type):
    if fill_type == 'All Fill Types':
        filtered_df = df_dates.copy()  
    else:
        filtered_df = df_dates[df_dates['Fill Type'] == fill_type].copy()  

    filtered_df.loc[:, 'YearMonth'] = filtered_df['Date'].dt.to_period('M').astype(str)  

    monthly_counts = filtered_df.groupby('YearMonth').size().reset_index(name='Total Entries')

    fig = px.bar(monthly_counts, x='YearMonth', y='Total Entries', 
                 labels={'YearMonth': 'Month', 'Total Entries': 'Total Entries'},
                 title=f'Total Fills per Month for {fill_type}')
    fig.update_layout(xaxis_tickangle=-45, yaxis_gridcolor='lightgrey', 
                      width=800, height=500) 
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
